## 五、问题与练习

### 1. 问题
#### 【问题一】 什么是fillna的前向/后向填充，如何实现？

In [1]:
import pandas as pd
import numpy as np

In [ ]:
pd.fill(method='bfill') 向前填充
ffill 向后填充

#### 【问题二】 下面的代码实现了什么功能？请仿照设计一个它的groupby版本。

In [6]:
s = pd.Series ([0, 1, 1, 0, 1, 1, 1, 0])
s1 = s.cumsum()
print(s1)
result = s.mul(s1)
result

0    0
1    1
2    2
3    2
4    3
5    4
6    5
7    5
dtype: int64


0    0
1    1
2    2
3    0
4    3
5    4
6    5
7    0
dtype: int64

不会

#### 【问题三】 如何计算组内0.25分位数与0.75分位数？要求显示在同一张表上。

In [7]:
df = pd.read_csv('data/Diamonds.csv').head()
df.describe(percentiles=[.05, .25, .75, .95])

,carat,depth,price
count,5.000000,5.000000,5.000000
mean,0.254000,60.780000,329.600000
std,0.043359,2.525272,4.505552
min,0.210000,56.900000,326.000000
5%,0.214000,57.480000,326.000000
25%,0.230000,59.800000,326.000000
50%,0.230000,61.500000,327.000000
75%,0.290000,62.400000,334.000000
95%,0.306000,63.120000,334.800000
max,0.310000,63.300000,335.000000


#### 【问题四】 既然索引已经能够选出某些符合条件的子集，那么filter函数的设计有什么意义？

不知道……感觉是更加规范

#### 【问题五】 整合、变换、过滤三者在输入输出和功能上有何异同？

整合的输入是函数，是对数据的一系列功能的集合，输出是对应函数下的值，mean……

变换是把相应的值变成xxx，输出是变换后的值

过滤的输入是布尔值，过滤的直接洗去了

#### 【问题六】 在带参数的多函数聚合时，有办法能够绕过wrap技巧实现同样功能吗？

wrap技巧我不大了解，这个问题等后面熟悉了再回答

### 2. 练习
#### 【练习一】： 现有一份关于diamonds的数据集，列分别记录了克拉数、颜色、开采深度、价格，请解决下列问题：

In [10]:
data = pd.read_csv('data/Diamonds.csv')
data.head()

,carat,color,depth,price
0,0.23,E,61.5,326
1,0.21,E,59.8,326
2,0.23,E,56.9,327
3,0.29,I,62.4,334
4,0.31,J,63.3,335


#### (a) 在所有重量超过1克拉的钻石中，价格的极差是多少？
#### (b) 若以开采深度的0.2\0.4\0.6\0.8分位数为分组依据，每一组中钻石颜色最多的是哪一种？该种颜色是组内平均而言单位重量最贵的吗？
#### (c) 以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，求每组中连续的严格递增价格序列长度的最大值。
#### (d) 请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）

In [11]:
price = data.query('carat > 1')['price']
price.max()-price.min()

17561

In [13]:
data = data.sort_values('depth')
data

,carat,color,depth,price
10377,1.09,J,43.0,4778
4518,1.00,G,43.0,3634
6341,1.00,G,44.0,4032
16857,1.43,I,50.8,6727
36503,0.30,E,51.0,945
...,...,...,...,...
53540,0.90,G,72.9,2691
46679,0.99,J,73.6,1789
41918,1.03,E,78.2,1262
52861,0.50,E,79.0,2579


In [31]:
bins = data['depth'].quantile(np.linspace(0,1,6)).tolist()

cuts = pd.cut(data['depth'],bins=bins)

# data.groupby(cuts)['color'].describe()

data['cuts'] = cuts

color_result = data.groupby('cuts')['color'].describe()

color_result

,count,unique,top,freq
cuts,,,,
"(43.0, 60.8]",11294,7,E,2259
"(60.8, 61.6]",11831,7,G,2593
"(61.6, 62.1]",10403,7,G,2247
"(62.1, 62.7]",10137,7,G,2193
"(62.7, 79.0]",10273,7,G,2000


In [45]:
data['average'] = data['price'] / data['carat']
color_result['top'] == [i[1] for i in data.groupby(['cuts'
                                ,'color'])['average'].mean().groupby(['cuts']).idxmax().values]


cuts
(43.0, 60.8]    False
(60.8, 61.6]    False
(61.6, 62.1]    False
(62.1, 62.7]     True
(62.7, 79.0]     True
Name: top, dtype: bool

#### (c) 以重量分组(0-0.5,0.5-1,1-1.5,1.5-2,2+)，按递增的深度为索引排序，求每组中连续的严格递增价格序列长度的最大值。

In [46]:
cuts = pd.cut(data['carat'],bins=[0,0.5,1,1.5,2,np.inf])
data['cuts'] = cuts

In [47]:
data.head()

,carat,color,depth,price,average,cuts
10377,1.09,J,43.0,4778,4383.486239,"(1.0, 1.5]"
4518,1.00,G,43.0,3634,3634.000000,"(0.5, 1.0]"
6341,1.00,G,44.0,4032,4032.000000,"(0.5, 1.0]"
16857,1.43,I,50.8,6727,4704.195804,"(1.0, 1.5]"
36503,0.30,E,51.0,945,3150.000000,"(0.0, 0.5]"


In [48]:
data.sort_values('cuts')

,carat,color,depth,price,average,cuts
52860,0.50,E,79.0,2579,5158.000000,"(0.0, 0.5]"
28561,0.30,H,61.6,675,2250.000000,"(0.0, 0.5]"
48807,0.50,D,61.6,2025,4050.000000,"(0.0, 0.5]"
46641,0.30,E,62.9,526,1753.333333,"(0.0, 0.5]"
28524,0.41,J,61.6,674,1643.902439,"(0.0, 0.5]"
...,...,...,...,...,...,...
25139,2.22,J,61.8,13703,6172.522523,"(2.0, inf]"
25146,2.20,I,61.8,13719,6235.909091,"(2.0, inf]"
26155,2.01,H,63.9,15467,7695.024876,"(2.0, inf]"
21630,2.01,J,62.8,9725,4838.308458,"(2.0, inf]"


In [56]:
def f(nums):
    if not nums:        
        return 0
    res = 1                            
    cur_len = 1                        
    for i in range(1, len(nums)):      
        if nums[i-1] < nums[i]:        
            cur_len += 1                
            res = max(cur_len, res)     
        else:                       
            cur_len = 1                 
    return res
for name,group in data.groupby('cuts'):
    group = group.sort_values(by='depth')
    s = group['price']
    print(name,f(s.tolist()))

(0.0, 0.5] 8
(0.5, 1.0] 7
(1.0, 1.5] 11
(1.5, 2.0] 6
(2.0, inf] 8


#### (d) 请按颜色分组，分别计算价格关于克拉数的回归系数。（单变量的简单线性回归，并只使用Pandas和Numpy完成）

#### 【练习二】：有一份关于美国10年至17年的非法药物数据集，列分别记录了年份、州（5个）、县、药物类型、报告数量，请解决下列问题：

In [58]:
data = pd.read_csv('data/Drugs.csv')
data.head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
1,2010,OH,ADAMS,Morphine,9
2,2010,PA,ADAMS,Methadone,2
3,2010,VA,ALEXANDRIA CITY,Heroin,5
4,2010,PA,ALLEGHENY,Hydromorphone,5


#### (a) 按照年份统计，哪个县的报告数量最多？这个县所属的州在当年也是报告数最多的吗？
#### (b) 从14年到15年，Heroin的数量增加最多的是哪一个州？它在这个州是所有药物中增幅最大的吗？若不是，请找出符合该条件的药物。

In [63]:
data.sort_values(by='DrugReports').head()

,YYYY,State,COUNTY,SubstanceName,DrugReports
0,2010,VA,ACCOMACK,Propoxyphene,1
9056,2013,KY,CARTER,Propoxyphene,1
9054,2013,KY,CARTER,Fentanyl,1
9053,2013,VA,CARROLL,Fentanyl,1
9051,2013,OH,CARROLL,Codeine,1


In [103]:
idx=pd.IndexSlice
for i in range(2010,2018):
    county = data.groupby(['COUNTY','YYYY']).sum().loc[idx[:,i],:].idxmax(0)[0][0]

    state1 = data.query('COUNTY == "%s"'%county)['State'].iloc[0]

    state2 = data.groupby(['State','YYYY']).sum().loc[idx[:,i],:].idxmax(0)[0][0]

    if state1==state2:
        print('在%d年，%s县的报告数最多，它所属的州%s也是报告数最多的'%(i,county,state1))
    else :
        print('在%d年，%s县的报告数最多，但它所属的州%s不是报告数最多的，%s州报告数最多'%(i,county,state1,state2))

在2010年，PHILADELPHIA县的报告数最多，它所属的州PA也是报告数最多的
在2011年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2012年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2013年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2014年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2015年，PHILADELPHIA县的报告数最多，但它所属的州PA不是报告数最多的，OH州报告数最多
在2016年，HAMILTON县的报告数最多，它所属的州OH也是报告数最多的
在2017年，HAMILTON县的报告数最多，它所属的州OH也是报告数最多的


In [128]:
data2 = data.query('(YYYY in [2014,2015]) & (SubstanceName == "Heroin" )').groupby(['YYYY','State']).sum()
name = (data2.loc[2015]-data2.loc[2014]).idxmax()[0]
data3 = data.query('(YYYY in [2014,2015]) & (State == "%s")'%name).groupby(['YYYY','SubstanceName']).sum()
name2 = (data3.loc[2015]-data3.loc[2014]).idxmax()[0]
name3 = (data3.loc[2015]/data3.loc[2014]).idxmax()[0]
name3

'Acetyl fentanyl'